In [1]:
import sys
import os
from itertools import chain
from collections import defaultdict
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision', 3)
pd.set_option('display.max_colwidth', -1)
import gseapy as gp
import goatools

goa = goatools.obo_parser.GODag('go-basic.obo')

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2018-01-12) 49,288 GO Terms


In [2]:
sys.path.insert(0, "/home/gstupp/projects/metaproteomics")
from metaproteomics import utils
#from metaproteomics.analysis import build_loci

BASE = 'out/'
grouped_loci = utils.load(os.path.join(BASE,"grouped_loci_filt_annot.pkl.gz"))

In [3]:
grouped_loci[0].annotations

{'Gene3D': {'G3DSA:1.10.285.10', 'G3DSA:3.40.192.10', 'G3DSA:3.40.50.720'},
 'PANTHER': {'PTHR11606', 'PTHR11606:SF4'},
 'PIRSF': {'PIRSF000185'},
 'PRINTS': {'PR00082'},
 'Pfam': {'PF00208', 'PF02812'},
 'SMART': {'SM00839'},
 'SUPERFAMILY': {'SSF51735', 'SSF53223'},
 'go': {'GO:0006520', 'GO:0016491', 'GO:0016639', 'GO:0055114'},
 'ipa': {'IPR006095', 'IPR006096', 'IPR006097', 'IPR014362', 'IPR016040'}}

In [39]:
# i need to map to slim, and can only do it with a gaf file
# so im making a fake gaf file with all go terms in it, then ill extract the go-> goslim mappings...
with open("all_go.gaf", "w") as f:
    print("!gaf-version: 2.0", file=f)
    for x in goa.values():
        go = x.id
        line = "\t".join(["UniProtKB", go, go, "", go, "zsrg", "sdg", 
                          "", "X", "", "", "protein", "taxon:9606", "20090118", "SGD", "", ""])
        print(line, file=f)    

In [57]:
go_cats = set(map(str.strip,open("idfile_slim.txt").readlines()))
go_cats

{'GO:0005575',
 'GO:0005623',
 'GO:0005737',
 'GO:0016020',
 'GO:0043226',
 'GO:0071944'}

In [ ]:
! ~/bin/owltools/OWLTools-Runner/bin/owltools go.obo --gaf all_go.gaf --map2slim --idfile idfile_slim.txt --write-gaf all_go.mapped.gaf

In [4]:
go_mapping = defaultdict(set)
go_mapping_df = pd.read_csv("all_go.mapped.gaf", comment="!", sep="\t", header=None)
for _, row in go_mapping_df.iterrows():
    go_mapping[row[1]].add(row[4])
go_mapping = dict(go_mapping)

In [6]:
list(go_mapping.items())[:5]

[('GO:0032195', {'GO:0005737', 'GO:0043226'}),
 ('GO:0032874', {'GO:0005623'}),
 ('GO:1902494', {'GO:0005575'}),
 ('GO:0098930', {'GO:0016020', 'GO:0071944'}),
 ('GO:0099114', {'GO:0005623', 'GO:0043226'})]

In [7]:
def get_go_count(loci):
    """
    if a protein has multiple go terms, give those go terms the number of spectrral counts/# Go terms
    """
    samples = {'H1-PelletTriflic-Fl-R1', 'H1-PelletTriflic-Fl-R2', 'H1-PelletTriflic-Fl-R3',
               'H1-SonicSupTriflic-V2-R1', 'H1-SonicSupTriflic-V2-R2', 'H1-SonicSupTriflic-V2-R3',
               'H1-wholetriflic-R1', 'H1-wholetriflic-R2', 'H1-wholetriflic-R3'}
    d = {s: defaultdict(int) for s in samples}
    for locus in loci:
        # map to slim
        go_terms = locus.annotations.get("go", {})
        this_go = set(chain(*[go_mapping[x] for x in go_terms if x in go_mapping]))
        # score
        for go in this_go:
            for sample,count in locus.quantification.items():
                d[sample][go]+=count/len(this_go)
    d = {k:dict(v) for k,v in d.items()}
    return d

In [8]:
df = pd.DataFrame(get_go_count(grouped_loci))
df.index = df.index.map(lambda x:goa[x].name)
df

,H1-PelletTriflic-Fl-R1,H1-PelletTriflic-Fl-R2,H1-PelletTriflic-Fl-R3,H1-SonicSupTriflic-V2-R1,H1-SonicSupTriflic-V2-R2,H1-SonicSupTriflic-V2-R3,H1-wholetriflic-R1,H1-wholetriflic-R2,H1-wholetriflic-R3
cellular_component,310.167,356.833,303.000,257.000,244.000,213.000,329.500,323.000,287.167
cell,7015.000,6850.500,6812.500,2305.833,2241.667,2263.833,6044.500,5795.500,5590.000
cytoplasm,6662.333,6718.167,7007.667,2053.833,2216.333,1872.667,6622.333,6347.667,5706.000
membrane,2058.000,2094.000,2037.167,2001.000,1895.500,1958.500,1819.167,1793.333,1655.667
organelle,3122.500,3099.000,3214.167,1225.000,1246.167,1268.667,2838.000,2853.500,2563.167
cell periphery,686.000,703.500,720.500,1008.333,994.333,1018.333,527.500,487.000,424.000


In [9]:
df_norm = df.div(df.sum(0))
df_norm

,H1-PelletTriflic-Fl-R1,H1-PelletTriflic-Fl-R2,H1-PelletTriflic-Fl-R3,H1-SonicSupTriflic-V2-R1,H1-SonicSupTriflic-V2-R2,H1-SonicSupTriflic-V2-R3,H1-wholetriflic-R1,H1-wholetriflic-R2,H1-wholetriflic-R3
cellular_component,0.016,0.018,0.015,0.029,0.028,0.025,0.018,0.018,0.018
cell,0.353,0.346,0.339,0.261,0.254,0.263,0.332,0.329,0.345
cytoplasm,0.336,0.339,0.349,0.232,0.251,0.218,0.364,0.361,0.352
membrane,0.104,0.106,0.101,0.226,0.214,0.228,0.100,0.102,0.102
organelle,0.157,0.156,0.160,0.138,0.141,0.148,0.156,0.162,0.158
cell periphery,0.035,0.035,0.036,0.114,0.113,0.118,0.029,0.028,0.026


In [13]:
membrane = {'membrane', 'cell periphery', 'organelle'}
non_membrane = set(df.index) - membrane

df.loc['mem_cp'] = df.loc[membrane].sum()
df.loc['non_mem'] = df.loc[non_membrane].sum()
df_mem = df.loc[['mem_cp', 'non_mem']]
df_mem

,H1-PelletTriflic-Fl-R1,H1-PelletTriflic-Fl-R2,H1-PelletTriflic-Fl-R3,H1-SonicSupTriflic-V2-R1,H1-SonicSupTriflic-V2-R2,H1-SonicSupTriflic-V2-R3,H1-wholetriflic-R1,H1-wholetriflic-R2,H1-wholetriflic-R3
mem_cp,5866.500,5896.500,5971.833,4234.333,4136.000,4245.500,5184.667,5133.833,4642.833
non_mem,13986.408,13924.418,14122.113,4615.206,4700.617,4347.976,12995.323,12465.147,11582.122


In [14]:
np.log(df_mem.loc['mem_cp']) - np.log(df_mem.sum())

H1-PelletTriflic-Fl-R1     -1.219
H1-PelletTriflic-Fl-R2     -1.212
H1-PelletTriflic-Fl-R3     -1.213
H1-SonicSupTriflic-V2-R1   -0.737
H1-SonicSupTriflic-V2-R2   -0.759
H1-SonicSupTriflic-V2-R3   -0.705
H1-wholetriflic-R1         -1.255
H1-wholetriflic-R2         -1.232
H1-wholetriflic-R3         -1.251
dtype: float64

In [15]:
np.log(df_mem.loc['mem_cp']) - np.log(df_mem.loc['non_mem'])

H1-PelletTriflic-Fl-R1     -0.869
H1-PelletTriflic-Fl-R2     -0.859
H1-PelletTriflic-Fl-R3     -0.861
H1-SonicSupTriflic-V2-R1   -0.086
H1-SonicSupTriflic-V2-R2   -0.128
H1-SonicSupTriflic-V2-R3   -0.024
H1-wholetriflic-R1         -0.919
H1-wholetriflic-R2         -0.887
H1-wholetriflic-R3         -0.914
dtype: float64

In [16]:
df.loc['ratio'] = np.log(df_mem.loc['mem_cp']) - np.log(df_mem.loc['non_mem'])
df

,H1-PelletTriflic-Fl-R1,H1-PelletTriflic-Fl-R2,H1-PelletTriflic-Fl-R3,H1-SonicSupTriflic-V2-R1,H1-SonicSupTriflic-V2-R2,H1-SonicSupTriflic-V2-R3,H1-wholetriflic-R1,H1-wholetriflic-R2,H1-wholetriflic-R3
cellular_component,310.167,356.833,303.000,257.000,244.000,213.000,329.500,323.000,287.167
cell,7015.000,6850.500,6812.500,2305.833,2241.667,2263.833,6044.500,5795.500,5590.000
cytoplasm,6662.333,6718.167,7007.667,2053.833,2216.333,1872.667,6622.333,6347.667,5706.000
membrane,2058.000,2094.000,2037.167,2001.000,1895.500,1958.500,1819.167,1793.333,1655.667
organelle,3122.500,3099.000,3214.167,1225.000,1246.167,1268.667,2838.000,2853.500,2563.167
cell periphery,686.000,703.500,720.500,1008.333,994.333,1018.333,527.500,487.000,424.000
ratio,-0.869,-0.859,-0.861,-0.086,-0.128,-0.024,-0.919,-0.887,-0.914
mem_cp,5866.500,5896.500,5971.833,4234.333,4136.000,4245.500,5184.667,5133.833,4642.833
non_mem,13986.408,13924.418,14122.113,4615.206,4700.617,4347.976,12995.323,12465.147,11582.122


In [17]:
df.to_csv("membrane_ratio.csv")

In [18]:
df.loc['cytoplasm_ratio'] = np.log(df.loc['cytoplasm']) - np.log(df.sum())
df.to_csv("cytoplasm_ratio.csv")
df

,H1-PelletTriflic-Fl-R1,H1-PelletTriflic-Fl-R2,H1-PelletTriflic-Fl-R3,H1-SonicSupTriflic-V2-R1,H1-SonicSupTriflic-V2-R2,H1-SonicSupTriflic-V2-R3,H1-wholetriflic-R1,H1-wholetriflic-R2,H1-wholetriflic-R3
cellular_component,310.167,356.833,303.000,257.000,244.000,213.000,329.500,323.000,287.167
cell,7015.000,6850.500,6812.500,2305.833,2241.667,2263.833,6044.500,5795.500,5590.000
cytoplasm,6662.333,6718.167,7007.667,2053.833,2216.333,1872.667,6622.333,6347.667,5706.000
membrane,2058.000,2094.000,2037.167,2001.000,1895.500,1958.500,1819.167,1793.333,1655.667
organelle,3122.500,3099.000,3214.167,1225.000,1246.167,1268.667,2838.000,2853.500,2563.167
cell periphery,686.000,703.500,720.500,1008.333,994.333,1018.333,527.500,487.000,424.000
ratio,-0.869,-0.859,-0.861,-0.086,-0.128,-0.024,-0.919,-0.887,-0.914
mem_cp,5866.500,5896.500,5971.833,4234.333,4136.000,4245.500,5184.667,5133.833,4642.833
non_mem,13986.408,13924.418,14122.113,4615.206,4700.617,4347.976,12995.323,12465.147,11582.122
cytoplasm_ratio,-1.785,-1.775,-1.747,-2.154,-2.076,-2.217,-1.703,-1.713,-1.738
